# Retail Analytics Notebook
Acest notebook încarcă datele demo, calculează indicatori și afișează grafice pentru trend și preț vs. volum.

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

root = Path("..").resolve().parent / "data" / "processed" if (Path.cwd() / ".." / "data" / "processed").exists() else Path("/mnt/data/retail_sales_project/data/processed")
fact = pd.read_csv(root/"fact_sales.csv", parse_dates=["OrderDate"])
dim_prod = pd.read_csv(root/"dim_product.csv")

# 1) Daily sales & MA28
daily = fact.groupby("OrderDate", as_index=False)["NetSales"].sum()
daily["MA28"] = daily["NetSales"].rolling(28, min_periods=7).mean()

plt.figure()
plt.plot(daily["OrderDate"], daily["NetSales"], label="Daily Sales")
plt.plot(daily["OrderDate"], daily["MA28"], label="MA28")
plt.title("Daily NetSales & 28-day Moving Average")
plt.xlabel("Date"); plt.ylabel("NetSales"); plt.legend(); plt.show()

# 2) Monthly totals
daily["YearMonth"] = daily["OrderDate"].dt.to_period("M").astype(str)
monthly = daily.groupby("YearMonth", as_index=False)["NetSales"].sum()

plt.figure()
plt.bar(monthly["YearMonth"], monthly["NetSales"])
plt.title("Monthly NetSales")
plt.xlabel("Year-Month"); plt.ylabel("NetSales"); plt.xticks(rotation=45, ha="right"); plt.show()

# 3) Price vs Qty (category weekly means)
df = fact[fact["Quantity"]>0].copy()
df["UnitPrice"] = df["NetSales"] / df["Quantity"]
df["Week"] = df["OrderDate"] - pd.to_timedelta(df["OrderDate"].dt.weekday, unit='D')
wk = df.groupby(["ProductID","Week"], as_index=False).agg(Qty=("Quantity","sum"), Price=("UnitPrice","mean"))
wk = wk.merge(dim_prod[["ProductID","Category"]], on="ProductID", how="left")
cat_week = wk.groupby(["Category","Week"], as_index=False).agg(Qty=("Qty","mean"), Price=("Price","mean"))

plt.figure()
plt.scatter(cat_week["Price"], cat_week["Qty"])
plt.title("Price vs Quantity (Category-level Weekly Means)")
plt.xlabel("Price (mean)"); plt.ylabel("Quantity (mean)"); plt.show()


## Forecast — Daily NetSales (SARIMAX, fallback seasonal naive)
Următorul cod antrenează un model SARIMAX cu sezonalitate săptămânală pe seria zilnică și face forecast pe 30 zile. Dacă librăria nu este disponibilă, folosește o metodă "seasonal naive" (media pe zi a săptămânii din ultimele 8 săptămâni).

In [ ]:

import pandas as pd, numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

root = Path("..").resolve().parent / "data" / "processed" if (Path.cwd() / ".." / "data" / "processed").exists() else Path("/mnt/data/retail_sales_project/data/processed")
daily = pd.read_csv(root/"fact_sales.csv", parse_dates=["OrderDate"]).groupby("OrderDate", as_index=False)["NetSales"].sum().sort_values("OrderDate")
daily = daily.set_index("OrderDate").asfreq("D").fillna(0.0)

history = daily["NetSales"]
n_forecast = 30

try:
    import statsmodels.api as sm
    model = sm.tsa.statespace.SARIMAX(history, order=(1,1,1), seasonal_order=(1,1,1,7), enforce_stationarity=False, enforce_invertibility=False)
    res = model.fit(disp=False)
    fc = res.get_forecast(steps=n_forecast)
    forecast_df = fc.summary_frame()[["mean","mean_ci_lower","mean_ci_upper"]]
    forecast_df.columns = ["yhat","yhat_lower","yhat_upper"]
    forecast_df.index.name = "Date"
except Exception:
    last_weeks = history.tail(56)
    weekday_means = last_weeks.groupby(last_weeks.index.weekday).mean()
    idx = pd.date_range(history.index[-1] + pd.Timedelta(days=1), periods=n_forecast, freq="D")
    yhat = [weekday_means[i.weekday()] for i in idx]
    forecast_df = pd.DataFrame({"yhat": yhat, "yhat_lower": np.nan, "yhat_upper": np.nan}, index=idx)
    forecast_df.index.name = "Date"

plt.figure()
plt.plot(history.index, history.values, label="History")
plt.plot(forecast_df.index, forecast_df["yhat"].values, label="Forecast")
plt.title("Daily NetSales Forecast (next 30 days)")
plt.xlabel("Date"); plt.ylabel("NetSales"); plt.legend(); plt.show()

forecast_df.head()
